# Module 40: SQLite Practice with Python

Author: Juliho David Castillo Colmenares

In this module, we will practice SQLite with Python. We will use the SQLite database to store the data and perform some operations on the data. We will use the `sqlite3` module in Python to interact with the SQLite database.

In [19]:
"""
Step 1: Import the necessary libraries and read the data from the CSV file
"""

import sqlite3
import pandas as pd
import os

current_directory = os.getcwd()
relative_path = "Customers.csv"
full_path = os.path.join(current_directory, relative_path)
df = pd.read_csv(full_path)

df.head()

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country,PurchaseUSD
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany,14861
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constituci?n 2222,M?xico D.F.,5021,Mexico,9900
2,3,Antonio Moreno Taquer?a,Antonio Moreno,Mataderos 2312,M?xico D.F.,5023,Mexico,7735
3,4,Around the Horn,Thomas Hardy,120 Hanover Sq.,London,WA1 1DP,UK,10295
4,5,Berglunds snabbk?p,Christina Berglund,Berguvsv?gen 8,Lule?,S-958 22,Sweden,6755


In [20]:
"""
Step 2: Create a SQLite database and save the data from the CSV file in the database
"""

# we define a 'now' variable to store the current time in seconds since epoch and in this way differentiate the databases created in different runs of the program
now = pd.Timestamp("now").value // 10**9  # current time in seconds since epoch

db_file = "Base" + str(now) + ".db"
db_path = os.path.join(current_directory, db_file)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create the Clientes table in SQLite
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS Clientes (
        CustomerID INTEGER PRIMARY KEY,
        CustomerName TEXT,
        ContactName TEXT,
        Address TEXT,
        City TEXT,
        PostalCode TEXT,
        Country TEXT
    )
"""
)

# Insert data into the table from the DataFrame
for _, row in df.iterrows():
    cursor.execute(
        """
        INSERT INTO Clientes (CustomerID, CustomerName, ContactName, Address, City, PostalCode, Country) 
        VALUES (?, ?, ?, ?, ?, ?, ?)""",
        (
            row["CustomerID"],
            row["CustomerName"],
            row["ContactName"],
            row["Address"],
            row["City"],
            row["PostalCode"],
            row["Country"],
        ),
    )

# Commit the changes
conn.commit()

In [21]:
""" 
Step 3: Save the database to a SQL file
"""

sql_file_name = "Base" + str(now) + ".sql"
sql_file_path = os.path.join(current_directory, sql_file_name)

with open(sql_file_path, "w") as f:
    for line in conn.iterdump():
        f.write("%s\n" % line)

conn.close()

print("Table 'Clientes' created and data inserted successfully.")

Table 'Clientes' created and data inserted successfully.


In [22]:
""" 
Step 4: Reconnect to the database and query the first 5 customers
"""

# Reconnect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Execute the query to get the first 5 customers
cursor.execute("SELECT * FROM Clientes LIMIT 5")
first_5_customers = cursor.fetchall()

# Convert the result to a DataFrame for better readability
first_5_customers_df = pd.DataFrame(
    first_5_customers,
    columns=[
        "CustomerID",
        "CustomerName",
        "ContactName",
        "Address",
        "City",
        "PostalCode",
        "Country",
    ],
)
print(first_5_customers_df)

# Close the connection
conn.close()

   CustomerID                        CustomerName         ContactName  \
0           1                 Alfreds Futterkiste        Maria Anders   
1           2  Ana Trujillo Emparedados y helados        Ana Trujillo   
2           3             Antonio Moreno Taquer?a      Antonio Moreno   
3           4                     Around the Horn        Thomas Hardy   
4           5                  Berglunds snabbk?p  Christina Berglund   

                         Address         City PostalCode  Country  
0                  Obere Str. 57       Berlin      12209  Germany  
1  Avda. de la Constituci?n 2222  M?xico D.F.       5021   Mexico  
2                 Mataderos 2312  M?xico D.F.       5023   Mexico  
3                120 Hanover Sq.       London    WA1 1DP       UK  
4                 Berguvsv?gen 8        Lule?   S-958 22   Sweden  
